In [17]:
"""
Notebook for streaming data from a microphone in realtime

audio is captured using pyaudio
then converted from binary data to ints using struct
then displayed using matplotlib

scipy.fftpack computes the FFT

if you don't have pyaudio, then run

>>> pip install pyaudio

note: with 2048 samples per chunk, I'm getting 20FPS
when also running the spectrum, its about 15FPS
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [18]:
import sys
import serial

serialPort = '/dev/cu.usbmodem14301'
baudRate = 115200
usbConnection = serial.Serial(serialPort, baudRate)

def commandArduino(string, blocking):
    print(string, end='')
    outputStr = string + '\n'
    outputStr = bytes(outputStr, 'utf-8')
    usbConnection.write(outputStr)
    
    if blocking==False:
        usbConnection.reset_input_buffer()
        print('')
    else:
        state = ''
        while state != 'Command complete':
            state = usbConnection.readline().decode().strip()
        print(' -- Command completed')

In [19]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK, exception_on_overflow = False)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    # volume
    delta = np.max(data_np) - np.min(data_np)
    delta_floored = max(int(delta ** 0.75), 0)
#     print(frame_count)
    
#     # compute FFT and update line
#     yf = fft(data_int)
#     yfft = np.abs(yf[0:CHUNK])  / (128 * CHUNK)
    
#     # max frequency
#     xMinOffset = 1
#     xMaxOffset = 2000
#     yfft_trimmed = yfft[xMinOffset:xMaxOffset]
#     maxIndex = list(yfft_trimmed).index(max(yfft_trimmed))
    
#     print(max(yfft_trimmed), xf[maxIndex+xMinOffset], delta)

    commandArduino(f'vibrate,{delta_floored}', False)
    
    # update figure canvas
    try:
#         line.set_ydata(data_np)
#         line_fft.set_ydata(yfft)
#         plt.show()
#         fig.canvas.draw()
#         fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started
vibrate,13
vibrate,26
vibrate,42
vibrate,22
vibrate,21
vibrate,26
vibrate,22
vibrate,17
vibrate,17
vibrate,16
vibrate,63
vibrate,63
vibrate,17
vibrate,20
vibrate,17
vibrate,19
vibrate,25
vibrate,17
vibrate,13
vibrate,13
vibrate,25
vibrate,19
vibrate,19
vibrate,24
vibrate,18
vibrate,22
vibrate,15
vibrate,16
vibrate,11
vibrate,19
vibrate,22
vibrate,28
vibrate,23
vibrate,23
vibrate,22
vibrate,21
vibrate,14
vibrate,18
vibrate,21
vibrate,21
vibrate,19
vibrate,29
vibrate,19
vibrate,16
vibrate,18
vibrate,17
vibrate,12
vibrate,16
vibrate,19
vibrate,27
vibrate,26
vibrate,21
vibrate,63
vibrate,43
vibrate,24
vibrate,63
vibrate,31
vibrate,22
vibrate,17
vibrate,16
vibrate,17
vibrate,19
vibrate,14
vibrate,17
vibrate,15
vibrate,15
vibrate,24
vibrate,18
vibrate,14
vibrate,63
vibrate,61
vibrate,19
vibrate,63
vibrate,62
vibrate,24
vibrate,16
vibrate,23
vibrate,19
vibrate,22
vibrate,15
vibrate,16
vibrate,16
vibrate,15
vibrate,21
vibrate,16
vibrate,11
vibrate,63
vibrate,46
vibrate,20
vibrat

KeyboardInterrupt: 